In [ ]:

import teehr

teehr.__version__

In [ ]:
from teehr.evaluation.spark_session_utils import create_spark_session

In [ ]:
spark = create_spark_session(
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin123",
    update_configs={
        "spark.hadoop.fs.s3a.aws.credentials.provider":  "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"
    }
)

In [ ]:
%%time
dir_path = "/data/temp_warehouse"

ev = teehr.Evaluation(
    spark=spark,
    dir_path=dir_path,
    check_evaluation_version=False,
)

In [ ]:
ev.set_active_catalog("remote")

ev.active_catalog

In [ ]:
from teehr import DeterministicMetrics as dm
from teehr import Signatures as s

SIM_METRICS = [
        s.Count(),
        s.Average(),
        dm.RelativeBias(add_epsilon=True),
        dm.NashSutcliffeEfficiency(add_epsilon=True),
        dm.KlingGuptaEfficiency(add_epsilon=True),
        dm.RootMeanStandardDeviationRatio(add_epsilon=True)
]
METRIC_COL_NAMES = [metric.output_field_name for metric in SIM_METRICS]
SIM_GROUPBY = ["primary_location_id", "configuration_name", "variable_name", "unit_name"]

In [ ]:
sdf = ev.metrics(table_name="sim_joined_timeseries").query(
    group_by=SIM_GROUPBY,
    order_by=SIM_GROUPBY,
    include_metrics=SIM_METRICS
).to_sdf()

In [ ]:
sdf.createTempView("metrics")

In [ ]:
sdf = ev.spark.sql("""
SELECT m.*, l.* FROM metrics m JOIN iceberg.teehr.locations l ON l.id = m.primary_location_id
""")
sdf = sdf.drop("id")

In [ ]:
sdf.show()

In [ ]:
ev.write.to_warehouse(source_data=sdf, table_name="sim_metrics_by_location", write_mode="create_or_replace")

In [ ]:
table_name = "sim_metrics_by_location"
properties = {
    "description": "Simulation metrics by location ID",
    "group_by": ", ".join(SIM_GROUPBY),
    "metrics": ", ".join(METRIC_COL_NAMES)
}

In [ ]:
for key, value in properties.items():
    ev.spark.sql(f"""
    ALTER TABLE iceberg.teehr.{table_name} SET TBLPROPERTIES ('{key}' = '{value}')
    """)

In [ ]:
spark.stop()